In [15]:
import scipy.io
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import math
import soundfile as sf
import os
from pydub import AudioSegment
from tqdm import tqdm
import random
random.seed(4)


In [16]:
# prendi il nome di tutti i files
audio_path = "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella"
audio_names = os.listdir(audio_path)
audio_names[:5]

['20190601_000000.WAV',
 '20190601_010000.WAV',
 '20190601_020000.WAV',
 '20190601_030000.WAV',
 '20190601_040000.WAV']

In [17]:
import json

category_info = {}
audio_info = {}
with open('utils\coords.json') as f:
    category_info = json.load(f)
with open('utils\inverted_coords.json') as f:
    audio_info = json.load(f)


In [18]:
des_segm_path = "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\segments"
os.makedirs(des_segm_path, exist_ok=True)
os.makedirs(os.path.join(des_segm_path, "train"), exist_ok=True)
os.makedirs(os.path.join(des_segm_path, "test"), exist_ok=True)

In [19]:
# per ogni categoria, prende tutti i file associati
# per ogni file, ne prende la durata
# se troppo corto (<3s) viene ignorato
# altrimenti viene salvato il time_frame da cui estrarre il segmento da 3s
# ogni segmento viene salvato sotto la categoria a cui appartiene
# tutti gli audio della stessa categoria vengono mescolati

target_dir = "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella"
extracted_info = {}
for category in category_info:
    files_info = category_info[category]
    extracted_info[category] = []
    for file in files_info:
        duration = file["duration"]
        if file["file_name"] not in audio_names or duration < 3:
            continue
        segments_start_time = [ file["start_time"] + i*3 for i in range(duration // 3) ]
        full_path = os.path.join(target_dir, file["file_name"])
        for segment in segments_start_time:
            extracted_info[category].append({"file_name": full_path, "start_time": segment})
    

In [20]:
extracted_info

{'Wind_': [{'file_name': 'E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella\\20190621_010000.WAV',
   'start_time': 94},
  {'file_name': 'E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella\\20190621_020000.WAV',
   'start_time': 80},
  {'file_name': 'E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella\\20190621_020000.WAV',
   'start_time': 83},
  {'file_name': 'E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella\\20190621_020000.WAV',
   'start_time': 86},
  {'file_name': 'E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella\\20190621_020000.WAV',
   'start_time': 157},
  {'file_name': 'E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella\\20190621_020000.WAV',
   'start_time': 568},
  {'file_name': 'E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella\\20190621_020000.WAV',
   'start_time': 571},
  {'file_name': 'E:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella\\20190621_020000.WAV',
   'start_time': 574},
  {'file_name': 'E:\\Giacomo\\Tovanella-20241110T12

In [21]:
for category in extracted_info:
    random.shuffle(extracted_info[category])

In [22]:
categories = list(extracted_info.keys())
for j in range(len(categories)):
    category = categories[j]
    os.makedirs(os.path.join(des_segm_path, "train", category), exist_ok=True)
    os.makedirs(os.path.join(des_segm_path, "test", category), exist_ok=True)
    all_category_audio = extracted_info[category]
    total_length = len(all_category_audio)
    threshold = 0.7
    train_length = int(total_length * threshold)
    test_length = total_length - train_length
    print(f"Loading {category} category... {j}/{len(categories)}")
    for i in tqdm(range(total_length)):
        split = "train" if i < train_length else "test"
        full_path = all_category_audio[i]["file_name"]
        file_name = full_path.split("\\")[-1]
        file_name = file_name[:-4]
        start_time = int(all_category_audio[i]["start_time"])
        export_path = os.path.join(des_segm_path, split, category, file_name+f"_{start_time}"+".wav")
        if os.path.exists(export_path):
            continue
        audio = AudioSegment.from_file(full_path, format="wav")
        segment = audio[start_time*1000:start_time*1000 + 3000]
        segment.export(export_path, format="wav")


Loading Wind_ category... 0/26


100%|██████████| 126/126 [00:14<00:00,  8.81it/s]


Loading Regulus ignicapilla_Common Firecrest category... 1/26


100%|██████████| 296/296 [00:21<00:00, 13.98it/s]


Loading Sylvia atricapilla_Eurasian Blackcap category... 2/26


...

100%|██████████| 40/40 [00:03<00:00, 12.21it/s]


Loading Dryocopus martius_Black Woodpecker category... 24/26


100%|██████████| 15/15 [00:02<00:00,  7.26it/s]


Loading Phylloscopus trochilus_Willow Warbler category... 25/26


100%|██████████| 1/1 [00:00<00:00, 20.49it/s]


In [23]:
# per ogni audio, per ogni segmento vengono indicate le classi al suo interno

audio_segments = {}
for category in extracted_info:
    for record in extracted_info[category]:
        file_name = record["file_name"].split("\\")[-1].upper()
        if file_name not in audio_segments:
            audio_segments[file_name] = {}
        if record["start_time"] // 3 not in audio_segments[file_name]:
            audio_segments[file_name][record["start_time"] // 3] = []
        audio_segments[file_name][record["start_time"] // 3].append(category)

for segments in audio_segments:
    audio_segments[segments] = dict(sorted(audio_segments[segments].items()))

with open("utils/segments_info.json", "w") as f:
    json.dump(audio_segments, f)

audio_segments

{'20190603_130000.WAV': {2: ['Wind_'],
  46: ['Regulus ignicapilla_Common Firecrest'],
  61: ['Wind_'],
  62: ['Wind_'],
  72: ['Wind_'],
  73: ['Wind_'],
  74: ['Wind_'],
  75: ['Wind_'],
  76: ['Wind_'],
  77: ['Wind_'],
  115: ['Troglodytes troglodytes_Eurasian Wren'],
  150: ['Wind_'],
  151: ['Wind_'],
  152: ['Wind_'],
  153: ['Wind_', 'Regulus ignicapilla_Common Firecrest'],
  154: ['Wind_'],
  155: ['Wind_'],
  156: ['Wind_'],
  157: ['Wind_'],
  158: ['Wind_'],
  177: ['Regulus ignicapilla_Common Firecrest']},
 '20190603_110000.WAV': {16: ['Regulus ignicapilla_Common Firecrest'],
  40: ['Troglodytes troglodytes_Eurasian Wren'],
  62: ['Fringilla coelebs_Common Chaffinch'],
  67: ['Troglodytes troglodytes_Eurasian Wren'],
  68: ['Fringilla coelebs_Common Chaffinch',
   'Troglodytes troglodytes_Eurasian Wren'],
  76: ['Troglodytes troglodytes_Eurasian Wren'],
  80: ['Sylvia atricapilla_Eurasian Blackcap'],
  85: ['Troglodytes troglodytes_Eurasian Wren'],
  91: ['Troglodytes trog